In [ ]:
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import numpy as np

In [ ]:
votes6 = os.path.join("Cleaned_data", "EP6_clean_data","votes_EP_6.csv")
votes7 = os.path.join("Cleaned_data", "EP7_clean_data","votes_EP_7.csv")
votes8= os.path.join("Cleaned_data", "EP8_clean_data","votes_EP_8.csv")
votes9 =os.path.join("Cleaned_data", "EP9_clean_data","votes_EP_9.csv")
votations6 = os.path.join("Cleaned_data", "EP6_clean_data","votations_EP_6.csv")
votations7 = os.path.join("Cleaned_data", "EP7_clean_data","votations_EP_7.csv")
votations8 = os.path.join("Cleaned_data", "EP8_clean_data","votations_EP_8.csv")
votations9 = os.path.join("Cleaned_data", "EP9_clean_data","votations_EP_9.csv")
mepinfo6 = os.path.join("Cleaned_data", "EP6_clean_data","mep_info_EP_6.csv")
mepinfo7 = os.path.join("Cleaned_data", "EP7_clean_data","mep_info_EP_7.csv")
mepinfo8 = os.path.join("Cleaned_data", "EP8_clean_data","mep_info_EP_8.csv")
mepinfo9 = os.path.join("Cleaned_data", "EP9_clean_data","mep_info_EP_9.csv")

In [ ]:
ep_votes6 = pd.read_csv(votes6)
ep_votes7 = pd.read_csv(votes7)
ep_votes8 = pd.read_csv(votes8)
ep_votes9 = pd.read_csv(votes9)
ep_mepinfo6 = pd.read_csv(mepinfo6)
ep_mepinfo7 = pd.read_csv(mepinfo7)
ep_mepinfo8 = pd.read_csv(mepinfo8)
ep_mepinfo9 = pd.read_csv(mepinfo9)
ep_votations6 = pd.read_csv(votations6)
ep_votations7 = pd.read_csv(votations7)
ep_votations8 = pd.read_csv(votations8)
ep_votations9 = pd.read_csv(votations9)

In [ ]:
votations_combined = votations_combined.astype({'FinalVote': 'Int64', 'Vote': 'Int64',"No":"Int64","Abs":"Int64",})


In [ ]:
ep_votations6 = ep_votations6.astype({'FinalVote': 'Int64', 'Vote': 'Int64',"No":"Int64","Abs":"Int64",})
ep_votations7 = ep_votations7.astype({'FinalVote': 'Int64', 'Vote': 'Int64',"No":"Int64","Abs":"Int64",})
ep_votations8 = ep_votations8.astype({'FinalVote': 'Int64', 'Vote': 'Int64',"No":"Int64","Abs":"Int64",})
ep_votations9 = ep_votations9.astype({'FinalVote': 'Int64', 'Vote': 'Int64',"No":"Int64","Abs":"Int64",})

In [ ]:
ep_votations6.Date = pd.to_datetime(ep_votations6.Date)
ep_votations7.Date = pd.to_datetime(ep_votations7.Date)
ep_votations8.Date = pd.to_datetime(ep_votations8.Date)
ep_votations9.Date = pd.to_datetime(ep_votations9.Date)

In [ ]:
votations_combined.Date = pd.to_datetime(votations_combined.Date)

In [ ]:
votations_combined.dtypes

In [ ]:
mepinfo_combined = pd.concat([ep_mepinfo6,ep_mepinfo7,ep_mepinfo8,ep_mepinfo9])

In [ ]:
ep_mepinfo6.Start =pd.to_datetime(ep_mepinfo6.Start)
ep_mepinfo7.Start = pd.to_datetime(ep_mepinfo7.Start)
ep_mepinfo8.Start = pd.to_datetime(ep_mepinfo8.Start)
ep_mepinfo9.Start = pd.to_datetime(ep_mepinfo9.Start)

In [ ]:
mepinfo_combined.Start = pd.to_datetime(mepinfo_combined.Start)

In [ ]:
ep_mepinfo6.End = pd.to_datetime(ep_mepinfo6.End)
ep_mepinfo7.End = pd.to_datetime(ep_mepinfo7.End)
ep_mepinfo8.End = pd.to_datetime(ep_mepinfo8.End)
ep_mepinfo9.End = pd.to_datetime(ep_mepinfo9.End)

In [ ]:
ep_votes6 = ep_votes6[['MepId',"VoteId","Vote"]]
ep_votes7 = ep_votes7[['MepId',"VoteId","Vote"]]
ep_votes8 = ep_votes8[['MepId',"VoteId","Vote"]]
ep_votes9 = ep_votes9[['MepId',"VoteId","Vote"]]

In [ ]:
ep_votes6


In [ ]:
ep_mepinfo7

In [ ]:
mepinfo_combined.End = pd.to_datetime(mepinfo_combined.End)

In [ ]:
votes_combined = pd.concat([ep_votes6,ep_votes7,ep_votes8,ep_votes9])

In [ ]:
votes_combined.EP_ID.value_counts()

In [ ]:
base_directory = os.path.join("Cleaned_data","Merged_dataset")
os.makedirs(base_directory, exist_ok=True)

In [ ]:
mepinfo_combined.to_csv(os.path.join(base_directory, "mep_info.csv"), index=False)
votes_combined.to_csv(os.path.join(base_directory, "votes.csv"), index=False)
votations_combined.to_csv(os.path.join(base_directory, "votations.csv"), index=False)

In [ ]:
def get_memberships_database(memberships_df):
    temp_df = memberships_df.copy()
    memberships_database = pd.DataFrame()
    memberships_database['MepId'] = temp_df['identifier']
    memberships_database['OrgId'] = temp_df['org_id']
    memberships_database['Classification'] = temp_df['membershipClassification']
    memberships_database['Start'] = temp_df['memberDuring.startDate']
    memberships_database['End'] = temp_df['memberDuring.endDate']
    memberships_database['Label'] = temp_df['org_label']
    return memberships_database

In [ ]:
def get_memberships_df(mep_df, org_df):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_membership, identifier) for identifier in mep_df['MepId']]
        results = [future.result() for future in as_completed(futures)]

    memberships_df = pd.concat(results, ignore_index=True)
    memberships_df.rename(columns={'organization': 'org_id'}, inplace=True)
    memberships_df = pd.merge(memberships_df, org_df, on='org_id', how='left')
    return memberships_df

In [ ]:
def get_membership(identifier):
    url = f"https://data.europarl.europa.eu/api/v2/meps/{identifier}?format=application%2Fld%2Bjson"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from URL: {url} - {e}")
        return pd.DataFrame()
    try:
        df = pd.json_normalize(data['data'], record_path=['hasMembership'],meta=['citizenship','bday','hasGender'])
    except ValueError as e:
        print(f"Error parsing JSON data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on JSON parsing error
    df1 = df[(df['membershipClassification'].isnull()) & (df['role']=="def/ep-roles/MEMBER_PARLIAMENT")]
    df2 = df[df['membershipClassification'].isin(["def/ep-entities/EU_POLITICAL_GROUP","def/ep-entities/NATIONAL_CHAMBER"])]
    df = pd.concat([df1,df2])
    try:
        df = df[['organization','membershipClassification','memberDuring.startDate','memberDuring.endDate','citizenship','bday','hasGender']].copy()
    except KeyError as e:
        df = df[['organization','membershipClassification','memberDuring.startDate','citizenship','bday','hasGender']].copy()
        df.loc[:, 'memberDuring.endDate'] = np.NaN
    df.loc[:,'identifier'] = identifier
    return df

In [ ]:
def get_epgs():
    url = 'https://data.europarl.europa.eu/api/v2/corporate-bodies?body-classification=EU_POLITICAL_GROUP&format=application%2Fld%2Bjson&offset=0'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from URL: {url} - {e}")
        return pd.DataFrame()
    try:
        df = pd.json_normalize(data['data'])
    except ValueError as e:
        print(f"Error parsing JSON data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on JSON parsing error
    df=df[['id','label']]
    epg_df = df.rename(columns={'id':'org_id',"label":"org_label"})
    return epg_df
def get_parties():
    url = 'https://data.europarl.europa.eu/api/v2/corporate-bodies?body-classification=NATIONAL_CHAMBER&format=application%2Fld%2Bjson&offset=0'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from URL: {url} - {e}")
        return pd.DataFrame()
    try:
        df = pd.json_normalize(data['data'])
    except ValueError as e:
        print(f"Error parsing JSON data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on JSON parsing error
    df=df[['id','label']]
    party_df = df.rename(columns={'id':'org_id',"label":"org_label"})
    return party_df
def generate_ep_df(ep_number):
    # Create a list of org_ids and org_labels based on the ep_number
    org_ids = [f'org/ep-{i}' for i in range(0, ep_number + 1)]
    org_labels = [f'EP{i}' for i in range(0, ep_number + 1)]

    # Create a DataFrame from the generated lists
    ep_df = pd.DataFrame({'org_id': org_ids, 'org_label': org_labels})
    return ep_df

In [ ]:
def get_memberships_database(memberships_df):
    temp_df = memberships_df.copy()
    memberships_database = pd.DataFrame()
    memberships_database['MepId'] = temp_df['identifier']
    memberships_database['OrgId'] = temp_df['org_id']
    memberships_database['Classification'] = temp_df['membershipClassification']
    memberships_database['Start'] = temp_df['memberDuring.startDate']
    memberships_database['End'] = temp_df['memberDuring.endDate']
    memberships_database['Label'] = temp_df['org_label']
    return memberships_database

In [ ]:
epg_df = get_epgs()
party_df = get_parties()
ep_df = generate_ep_df(9)
org_df = pd.concat([epg_df, party_df,ep_df], ignore_index=True)

In [ ]:
ep_mepinfo7

In [ ]:
membership6_df = get_memberships_df(ep_mepinfo6,org_df)
membership7_df = get_memberships_df(ep_mepinfo7,org_df)
membership8_df = get_memberships_df(ep_mepinfo8,org_df)
membership9_df = get_memberships_df(ep_mepinfo9,org_df)

In [ ]:
membership6_df = get_memberships_df(ep_mepinfo6,org_df)

In [ ]:
membership6_df

In [ ]:
membership8_df = get_memberships_df(ep_mepinfo8,org_df)


In [ ]:
membership8_df

In [ ]:
membership7_df = get_memberships_df(ep_mepinfo7,org_df)

In [ ]:
membership9_df = get_memberships_df(ep_mepinfo9,org_df)

In [ ]:
membership9_df

In [ ]:
mem9db = get_memberships_database(membership9_df)
mem6db = get_memberships_database(membership6_df)
mem7db = get_memberships_database(membership7_df)
mem8db = get_memberships_database(membership8_df)